In [1]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math
%matplotlib inline


C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
rolling_mask_two = [-i for i in range(1,2)]+[i for i in range(1,2)]
rolling_mask_four = [-i for i in range(1,3)]+[i for i in range(1,3)]
rolling_mask_six = [-i for i in range(1,4)]+[i for i in range(1,4)]
rolling_mask_eight = [-i for i in range(1,5)]+[i for i in range(1,5)]
rolling_mask_ten = [-i for i in range(1,6)]+[i for i in range(1,6)]

In [3]:
train = pd.read_csv('../data/public_raw.train.csv')

test = pd.read_csv('../data/public_raw.test.csv')

train['is_train']=1
test['is_train']=0

df = pd.concat([train, test],sort=False)

rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)

In [4]:
#修正
#计算偏差率的辅助列
for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
    df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
    df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']
    
    
#标记包含异常值的记录
df['is_abnormal']=0

for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
    df.loc[df[c+'_exception_ratio'] > 1.6 , 'is_abnormal'] = 1

    


In [5]:
#训练集中异常记录30条
df[df['is_train']==1][df['is_abnormal']==1][['ID','board_t','light_strength','I_A','I_B','I_C','V_A','V_B','V_C','P_avg','y']].shape

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(30, 11)

In [6]:
#训练集中异常记录34条
df[df['is_train']==0][df['is_abnormal']==1][['ID','board_t','light_strength','I_A','I_B','I_C','V_A','V_B','V_C','P_avg','y']].shape

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(34, 11)

In [7]:
#异常值由滚动平均值替代（前后各4个点的平均值）
static=1
for idx, line in df.iterrows():
    for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
        if line[c+'_exception_ratio']>1.6:
            print(str(line[c]) + ' is abnormal as value of ' + c)
            print('Mark for abnormal records: ' + str(line['is_abnormal']))
            line.loc[c] = line[c+'_avg_sequence']
            print('Has been replaced by '+str(line[c+'_avg_sequence'])) 
            static += 1
            print(static)

645.39 is abnormal as value of I_B
Mark for abnormal records: 1.0
Has been replaced by 2.85625
2
65382.0 is abnormal as value of V_A
Mark for abnormal records: 1.0
Has been replaced by 722.25
3
65498.0 is abnormal as value of V_C
Mark for abnormal records: 1.0
Has been replaced by 677.125
4
6.78 is abnormal as value of I_B
Mark for abnormal records: 1.0
Has been replaced by 2.56
5
6.68 is abnormal as value of I_A
Mark for abnormal records: 1.0
Has been replaced by 2.22125
6
640.77 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 241.31999999999994
7
65402.0 is abnormal as value of V_B
Mark for abnormal records: 1.0
Has been replaced by 24870.625
8
640.77 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 241.33249999999998
9
65402.0 is abnormal as value of V_B
Mark for abnormal records: 1.0
Has been replaced by 24871.75
10
6.68 is abnormal as value of I_A
Mark for abnormal records: 1.0
Has been replaced by 2.2449999999999997
1

65505.0 is abnormal as value of V_B
Mark for abnormal records: 1.0
Has been replaced by 664.875
132
65491.0 is abnormal as value of V_C
Mark for abnormal records: 1.0
Has been replaced by 643.875
133
1.38 is abnormal as value of I_B
Mark for abnormal records: 1.0
Has been replaced by 0.35125
134
0.74 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 0.28125000000000006
135
32.91 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 7.88625
136
8.86 is abnormal as value of I_B
Mark for abnormal records: 1.0
Has been replaced by 3.3025
137
8.39 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 3.185
138
7.99 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 2.12625
139
7.49 is abnormal as value of I_B
Mark for abnormal records: 1.0
Has been replaced by 1.6087500000000001
140
8.05 is abnormal as value of I_C
Mark for abnormal records: 1.0
Has been replaced by 1.85875
141
5.4

In [8]:
#前二后二
next_one = []
prev_one = []
next_id = []
prev_id = []

second_next_one = []
second_prev_one = []

df_len = df.shape[0]

i_y =df.columns.get_loc("y")

def get_prev_nn_index(cur_i):
    prev_i = cur_i-1
    while(prev_i>=0 and pd.isnull(df.iat[prev_i,i_y])):
        prev_i-=1
    return prev_i

def get_next_nn_index(cur_i):
    prev_i = cur_i+1
    while(prev_i<df_len and pd.isnull(df.iat[prev_i,i_y])):
        prev_i+=1
    return prev_i

for i in range(df_len):
    f_pre_i=get_prev_nn_index(i)
    if(f_pre_i)<0:
        prev_one.append(np.nan)
        prev_id.append(0)
    else:
        prev_one.append(df.iat[f_pre_i,i_y])
        prev_id.append(f_pre_i)
        
    s_pre_i=get_prev_nn_index(f_pre_i)
    if (s_pre_i)<0:
        second_prev_one.append(np.nan)
    else:
        second_prev_one.append(df.iat[s_pre_i,i_y])
    
    f_next_i=get_next_nn_index(i)
    if(f_next_i<df_len):
        next_one.append(df.iat[f_next_i,i_y])
        next_id.append(f_next_i)
    else:
        next_one.append(np.nan)
        next_id.append(df_len)
    
    s_next_i=get_next_nn_index(f_next_i)
    if(s_next_i<df_len):
        second_next_one.append(df.iat[s_next_i,i_y])
    else:
        second_next_one.append(np.nan)
        

df['next_value'] = next_one
df['prev_value'] = prev_one
df['avg_value'] = np.nanmean([df['next_value'], df['prev_value']],axis=0)

df.drop(['next_value','prev_value'],1,inplace=True)

# df.drop(['P_A','P_B','P_C','P_avg'],1,inplace=True)
df['P_A_cor']=df['I_A']*df['V_A']
df['P_B_cor']=df['I_B']*df['V_B']
df['P_C_cor']=df['I_C']*df['V_C']
df['P_avg_cor']=1/3*(df['P_A_cor']+df['P_B_cor']+df['P_C_cor'])

In [9]:
#拆分数据

train_data = df[df['is_train']==1]
test_data = df[df['is_train']==0]
len(train_data), len(test_data)

(9000, 8409)

In [10]:
#准备Test集结果
df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

In [12]:
train_data.head()

,ID,board_t,env_t,light_strength,efficiency,efficiency_A,efficiency_B,efficiency_C,V_A,V_B,...,V_B_avg_sequence,V_B_exception_ratio,V_C_avg_sequence,V_C_exception_ratio,is_abnormal,avg_value,P_A_cor,P_B_cor,P_C_cor,P_avg_cor
2,10,-19.14,-17.4,34,80.55,106.32,16.98,118.36,729,709,...,597.666667,0.186280,603.666667,0.200994,0,1.692575,976.86,155.98,1087.50,740.113333
3,11,-18.73,-17.3,30,99.90,139.00,21.20,139.51,728,717,...,615.285714,0.165312,621.285714,0.168544,0,1.706770,1128.40,172.08,1132.56,811.013333
4,12,-17.54,-17.0,41,82.48,114.86,14.91,117.66,731,722,...,628.750000,0.148310,634.875000,0.134082,0,2.031615,1279.25,166.06,1310.40,918.570000
6,14,-15.43,-16.6,53,73.98,101.72,15.55,104.67,730,727,...,721.500000,0.007623,725.375000,0.000862,0,2.253939,1474.60,225.37,1517.34,1072.436667
7,15,-14.60,-16.3,65,64.62,86.86,13.09,93.92,727,729,...,723.375000,0.007776,725.125000,0.003965,0,2.575187,1548.51,233.28,1674.40,1152.063333


In [13]:
#Train集去重
print(train_data.shape)
train_data = train_data.drop_duplicates(train_data.columns.drop(['ID','avg_value','P_A_cor','P_B_cor','P_C_cor','P_avg_cor','V_B_avg_sequence','V_B_exception_ratio','V_A_avg_sequence','V_A_exception_ratio','V_C_avg_sequence','V_C_exception_ratio','I_B_avg_sequence','I_B_exception_ratio','I_A_avg_sequence','I_A_exception_ratio','I_C_avg_sequence','I_C_exception_ratio',]), keep='first')
print(train_data.shape)

(9000, 40)
(8918, 40)


In [14]:
# 生成数据
def generate_train_data(train_data, test_data, poly=False, select=False):
    y = train_data['y']
    X = train_data.drop(['y','ID','is_train'], axis=1)
    sub_data = test_data.drop(['y','ID','is_train'], axis=1)
    
    polynm = None
    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        polynm = PolynomialFeatures(degree=2, interaction_only=True)
        X = polynm.fit_transform(X)
        sub_data = polynm.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
    sm = None
    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        sub_data = sm.transform(sub_data)
        
    return X_train, X_test, y_train, y_test, sub_data, sm, polynm

In [15]:
X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=False, select=False)

In [16]:
all_X_train = np.concatenate([X_train, X_test])
all_y_train = np.concatenate([y_train, y_test])

In [17]:
# 生成数据
# def generate_train_data(train_data, test_data, poly=False, select=False):
#     y = train_data['y']
#     X = train_data.drop(['y','ID','is_train'], axis=1)
#     sub_data = test_data.drop(['y','ID','is_train'], axis=1)
    
#     polynm = None
#     if poly:
#         from sklearn.preprocessing import PolynomialFeatures
#         polynm = PolynomialFeatures(degree=2, interaction_only=True)
#         X = polynm.fit_transform(X)
#         sub_data = polynm.transform(sub_data)
        
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
#     sm = None
#     if select:
#         from sklearn.feature_selection import SelectFromModel
#         sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
#         X_train = sm.fit_transform(X_train, y_train)
#         X_test = sm.transform(X_test)
#         sub_data = sm.transform(sub_data)
        
#     return X_train, X_test, y_train, y_test, sub_data, sm, polynm

def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))
#  定义交叉验证函数  
def cross_validation_test(models, train_X_data, train_y_data, cv=5):
    model_name, mse_avg, score_avg = [], [], []
    for i, model in enumerate(models):
        print(i + 1,'- Model:', str(model).split('(')[0])
        model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
        nmse = cross_val_score(model, train_X_data[i], train_y_data[i], cv=cv, scoring='neg_mean_squared_error')
        avg_mse = np.average(-nmse)
        scores = cal_score(-nmse)
        avg_score = np.average(scores)
        mse_avg.append(avg_mse)
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average XGB - MSE:', avg_mse, ' - Score:', avg_score, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    return res

# def add_newid(df):
#     ID = df["ID"]
#     df["new_id"]=(np.mod(ID,205))
#     return df
# def add_avg(df):
#     array = np.array(df["P_avg"])
#     newarray=[]
#     num = 0
#     for i in np.arange(len(array)):
#         for j in np.arange(10):
#             if i<10:
#                 num = (array[j-1]+array[j-2]+array[j-3])/3
#             if i>=10:
#                 num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
#         newarray.append(num)
#     df["old_SoCalledSF_P_avg"] = newarray
#     return df

In [18]:
xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=321, n_jobs=8)
xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=456, n_jobs=8)
xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=789, n_jobs=8)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=123,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=456,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=800, max_features='sqrt', random_state=7, n_jobs=8)
forest2 = RandomForestRegressor(n_estimators=900, max_features='log2', random_state=9, n_jobs=8)
forest3 = RandomForestRegressor(n_estimators=900, max_features='sqrt', random_state=11, n_jobs=8) 

lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5, n_jobs=8) 
lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7, n_jobs=8)
lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=9, n_jobs=8)

# xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=2, n_jobs=8)
# xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=3, n_jobs=8)
# xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=4, n_jobs=8)

# gbdt1 = GradientBoostingRegressor(n_estimators=500, max_depth=3, max_features='sqrt', random_state=2)
# gbdt2 = GradientBoostingRegressor(n_estimators=400, max_depth=3, max_features='sqrt', random_state=3)
# gbdt3 = GradientBoostingRegressor(n_estimators=500, max_depth=4, max_features='log2', random_state=4)

# forest1 = RandomForestRegressor(n_estimators=300, max_features='sqrt', random_state=2, n_jobs=8)
# forest2 = RandomForestRegressor(n_estimators=300, max_features='log2', random_state=3, n_jobs=8)
# forest3 = RandomForestRegressor(n_estimators=600, max_features='sqrt', random_state=4, n_jobs=8) 

# lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=2, n_jobs=8) 
# lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=3, n_jobs=8)
# lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=4, n_jobs=8)

In [19]:
#Cross Validation

cross_validation_test(
    models=[    
        xgbt1, xgbt2, xgbt3,
        gbdt1, gbdt2, gbdt3,
        forest1, forest2, forest3,
        lgb1, lgb2, lgb3
    ],
    train_X_data=[
        all_X_train, all_X_train, all_X_train, all_X_train,
        all_X_train, all_X_train, all_X_train, all_X_train,
        all_X_train, all_X_train, all_X_train, all_X_train
    ],
    train_y_data=[
        all_y_train, all_y_train, all_y_train, all_y_train,
        all_y_train, all_y_train, all_y_train, all_y_train,
        all_y_train, all_y_train, all_y_train, all_y_train
    ]
)

1 - Model: XGBRegressor
MSE: [0.01362767 0.03184425 0.01639666 0.08434111 0.01868022]
Score: [0.89546551 0.84857258 0.88648597 0.77494425 0.87975847]
Average XGB - MSE: 0.03297798205217808  - Score: 0.8570453566872016 

2 - Model: XGBRegressor


KeyboardInterrupt: 

In [20]:
regrs = [
    xgbt1, gbdt1, forest1, lgb1,
    xgbt2, gbdt2, forest2, lgb2,
    xgbt3, gbdt3, forest3, lgb3
]

In [21]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # X: 原始训练集, y: 原始训练集真实值, predict_data: 原始待预测数据
    def fit_predict(self, X, y, predict_data):
        X = np.array(X)
        y = np.array(y)
        T = np.array(predict_data)

        folds = list(KFold(n_splits=self.n_splits, shuffle=False, random_state=2018).split(X, y))
        
        # 以基学习器预测结果为特征的 stacker的训练数据 与 stacker预测数据
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_predict = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, regr in enumerate(self.base_models):
            print(i + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((T.shape[0], self.n_splits))
            
            for j, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train, y_train, X_test, y_test = X[train_idx], y[train_idx], X[test_idx], y[test_idx]
                print ('Fit fold', (j+1), '...')
                regr.fit(X_train, y_train)
                    y_pred = regr.predict(X_test)                
                S_train[test_idx, i] = y_pred
                S_predict_i[:, j] = regr.predict(T)
            
            S_predict[:, i] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [22]:
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(all_X_train, all_y_train, sub_data)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
5 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
6 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
7 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
8 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
9 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
10 Base model: GradientBoostingRegre

In [23]:
df_result['score'] = pred_stack

index = df_result[df_result['ID'].isin(special_missing_ID)].index
df_result.loc[index, 'score'] = 0.379993053

In [25]:
df_result['score'].describe()

count    8409.000000
mean        5.696030
std         3.460621
min        -0.454089
25%         2.515640
50%         5.704791
75%         8.890597
max        12.150429
Name: score, dtype: float64

In [26]:
df_result.to_csv('../result/081303_08724.csv', index=False, header=False)